In [55]:
from api_key import get_api_key
import google.generativeai as genai
from vpn_control import windscribe
import time
import re
import random
import datetime
import pandas as pd
import matplotlib
import numpy as np
import seaborn 
import requests


In [73]:
windscribe("connect", "Atlanta")
time.sleep(5)

In [ ]:
passw = ""

In [77]:
def setup_prompt_env(passw, extra_instruct):


  genai.configure(api_key=get_api_key(passw))

  # Set up the model
  generation_config = {
    "temperature": 0,
    "top_p": 0.95,
    "top_k": 0,
    "max_output_tokens": 300,
  }

  safety_settings = [
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
  ]

  system_instruction = f"""You must modify the log template such that it coveys the same infromation but is structred differently/uses different words. You can add or remove <*> wildcard which will contain variables later. Only use sensible replacements in the context.  Additonally {extra_instruct}. Finally you will be given a change_level token at the start of the prom which tells you how much of the original log should be modified from 0 to 10, where 0 is no change and 10 is every token/word and order can be changed. Pas the log with a begin and end token <B> <E>, dont give addtional text. Find am example below
  
  ORGINAL: Node <*> experienced data loss at <*>
  change_level 0: Node <*> experienced data loss at <*>
  change_level 5: <*> node data loss at <*>  
  change_level 10: HDFS node number <*> lost data due to unkown falure at time <*> """

  model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                                generation_config=generation_config,
                                system_instruction=system_instruction,
                                safety_settings=safety_settings)

  convo = model.start_chat(history=[
  ])

  return convo


In [58]:

def augment_template_LLM(message, convo, change_Level, extra_instruct):
    message =  str(change_Level) + "\n" + message 
    convo.send_message(message)
    suggestion = convo.last.text
    print(suggestion)
    return suggestion


In [59]:
def setup_LLM_aug(passw, extra_instruct, message, change_level):
    convo_create = setup_prompt_env(passw, extra_instruct)
    augment_template_LLM(message, convo_create, change_level, extra_instruct)

In [60]:
setup_LLM_aug(passw, "make it sound like it was written by a pirate", "Computer <*> overheated and HDD failed at <*>", 10)

<B> Shiver me timbers! Thar be a <*> that be hotter than a kraken's breath, and the treasure chest be sunk at <*>! <E> 



In [61]:
setup_LLM_aug(passw, "make it very robotic in language", "Computer <*> overheated and HDD failed at <*>", 10)

<B> Unit <*> experienced HDD malfunction following critical thermal event at <*> <E> 



In [62]:
setup_LLM_aug(passw, "make it very verbose", "Computer <*> overheated and HDD failed at <*>", 10)

<B> At the time instance of <*> , the hard disk drive of the computing device experienced a catastrophic failure due to exceeding thermal operating parameters. <E> 



In [78]:
setup_LLM_aug(passw, "use terms related to HDFS", "Computer <*> overheated and HDD failed at <*>", 0)

<B> Computer <*> overheated and HDD failed at <*> <E> 



In [79]:
setup_LLM_aug(passw, "use terms related to HDFS", "Computer <*> overheated and HDD failed at <*>", 5)

<B> <*> HDD failure due to overheat in machine <*> <E> 



In [80]:
setup_LLM_aug(passw, "use terms related to HDFS", "Computer <*> overheated and HDD failed at <*>", 10)

<B> HDFS datanode <*> suffered a disk failure due to overheating at time <*> <E> 



In [81]:
windscribe("disconnect")

In [82]:
asD

NameError: name 'asD' is not defined

In [ ]:
def generate_random_values_LLM(aug_template):

    placeholders = re.findall(r"<\*>", template)
    
    for placeholder in placeholders:
        prompt = f"Generate a contextually appropriate value for the placeholder in this context: {template}"
        value = get_gemini_suggestion(aug_template)
        
        template = template.replace("<*>", value, 1)
    
    return template


In [ ]:
windscribe("disconnect")